가장 성능이 좋은 모델로 판단되는 모델에 대한 정보를 .pt 파일로 저장되어 다음 최종 직무 추천 알고리즘에서 사용될 수 있도록 하였습니다

In [ ]:
import torch
import torch.nn as nn  # nn 모듈 추가
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoTokenizer, AutoModel  # AutoModel 추가
import os

# ========================================================================================
# [Model Architecture] 독립 실행을 위해 모델 클래스 직접 정의
# ========================================================================================
class ReviewRatingRegressor(nn.Module):
    def __init__(self, n_targets=6):
        super(ReviewRatingRegressor, self).__init__()
        self.bert = AutoModel.from_pretrained('klue/bert-base') # 모델명 하드코딩 (학습과 동일)
        self.drop = nn.Dropout(p=0.3)
        self.out = nn.Linear(self.bert.config.hidden_size, n_targets)

    def forward(self, input_ids, attention_mask):
        _, pooler_output = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            return_dict=False
        )
        output = self.drop(pooler_output)
        return self.out(output), pooler_output

# ========================================================================================
# [Preprocessing] 전처리 함수 직접 정의
# ========================================================================================
def local_preprocess_dataframe(df):
    # 컬럼명 공백 제거
    df.columns = [str(col).strip() for col in df.columns]

    # 컬럼 매핑 (한글 -> 영문)
    rename_map = {
        '구분': 'job_role', '기업명': 'company',
        '복지 및 급여': 'salary', '업무와 삶의 균형': 'wlb', '사내문화': 'culture',
        '경영진': 'leadership', '승진 기회': 'promotion', '종합평점': 'overall'
    }
    available_cols = set(df.columns)
    valid_rename = {k:v for k,v in rename_map.items() if k in available_cols}
    df = df.rename(columns=valid_rename)

    # 텍스트 결합 (장점 + 단점 + 요약 -> full_text)
    df['full_text'] = (
        df['장점'].fillna('').astype(str) + " " +
        df['단점'].fillna('').astype(str) + " " +
        df['요약'].fillna('').astype(str)
    )
    return df

# ========================================================================================
# [Inference Class] 추천 시스템
# ========================================================================================
class AIRecommender:
    def __init__(self, model_path, data_path):
        # 1. 디바이스 설정
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        print(f"[Inference] 사용 디바이스: {self.device}")

        # 2. 토크나이저 로드
        self.tokenizer = AutoTokenizer.from_pretrained('klue/bert-base')

        # 3. 모델 아키텍처 로드 및 가중치 복원
        try:
            # train 모듈 의존성 제거 -> 내부 클래스 사용
            self.model = ReviewRatingRegressor(n_targets=6)

            # [수정] 코랩 경로 호환성 강화
            if not os.path.exists(model_path):
                 # 만약 입력된 경로에 없으면 코랩 기본 경로 확인
                if os.path.exists('/content/review_rating_model.pt'):
                    print(f"⚠️ 지정된 경로({model_path})에 파일이 없어 '/content/review_rating_model.pt'를 대신 사용합니다.")
                    model_path = '/content/review_rating_model.pt'

            self.model.load_state_dict(torch.load(model_path, map_location=self.device))
            self.model.to(self.device)
            self.model.eval()
            print(f"✅ 학습된 모델 로드 완료! (경로: {model_path})")

        except FileNotFoundError:
            raise FileNotFoundError(f"❌ 모델 파일({model_path})을 찾을 수 없습니다. 학습 코드를 먼저 실행해 주세요.")

        # 4. 추천 후보군 데이터 준비
        self.data_path = data_path
        self.df = self._load_and_preprocess_data()

        # 5. 전체 데이터 임베딩 생성
        print("\n[Inference] 전체 기업 데이터의 임베딩(Vector)을 생성 중입니다...")
        self.company_embeddings = self._get_embeddings(self.df['full_text'].tolist())
        print(f" -> 임베딩 생성 완료. (Shape: {self.company_embeddings.shape})")

    def _load_and_preprocess_data(self):
        # 1. 파일 존재 여부 및 경로 보정
        if not os.path.exists(self.data_path):
            # 코랩 환경 예외 처리: 엑셀 파일 우선 탐색
            if os.path.exists('/content/통합_리뷰_기업중복제거.xlsx'):
                 self.data_path = '/content/통합_리뷰_기업중복제거.xlsx'
            # CSV 파일 (변환된 것) 차선 탐색
            elif os.path.exists('/content/통합_리뷰_기업중복제거.xlsx - Sheet1.csv'):
                 self.data_path = '/content/통합_리뷰_기업중복제거.xlsx - Sheet1.csv'
            else:
                raise FileNotFoundError(f"데이터 파일({self.data_path})을 찾을 수 없습니다.")

        print(f"[Inference] 데이터 로드 중... ({self.data_path})")

        # 2. 확장자에 따른 로드 (Excel vs CSV)
        if self.data_path.endswith('.xlsx') or self.data_path.endswith('.xls'):
            try:
                df = pd.read_excel(self.data_path)
            except Exception as e:
                raise ValueError(f"엑셀 파일을 읽는 중 오류가 발생했습니다. (openpyxl 설치 확인 필요): {e}")
        else:
            # CSV로 시도하되 실패하면 Excel로 재시도 (확장자가 잘못된 경우 대비)
            try:
                df = pd.read_csv(self.data_path)
            except:
                try:
                    df = pd.read_excel(self.data_path)
                except Exception as e:
                    raise ValueError(f"파일을 읽을 수 없습니다: {e}")

        # 내부 전처리 함수 사용
        df = local_preprocess_dataframe(df)
        return df

    def _get_embeddings(self, text_list):
        embeddings = []
        batch_size = 32

        with torch.no_grad():
            for i in range(0, len(text_list), batch_size):
                batch_texts = text_list[i:i+batch_size]

                encoded = self.tokenizer.batch_encode_plus(
                    batch_texts,
                    max_length=128,
                    padding='max_length',
                    truncation=True,
                    return_tensors='pt'
                )

                input_ids = encoded['input_ids'].to(self.device)
                mask = encoded['attention_mask'].to(self.device)

                # 모델 통과 -> 임베딩 추출
                _, pooler_output = self.model(input_ids, mask)
                embeddings.append(pooler_output.cpu().numpy())

        return np.vstack(embeddings)

    def recommend(self, user_query, top_k=3):
        print(f"\n" + "="*60)
        print(f"🔍 사용자 입력: '{user_query}'")
        print("="*60)

        query_vec = self._get_embeddings([user_query])
        sim_scores = cosine_similarity(query_vec, self.company_embeddings).flatten()
        top_indices = sim_scores.argsort()[-top_k:][::-1]

        print(f"🤖 AI 추천 결과 (Top {top_k}):")

        rank = 1
        for idx in top_indices:
            row = self.df.iloc[idx]
            sim = sim_scores[idx]

            c_name = row.get('company', 'Unknown')
            role = row.get('job_role', 'Unknown')
            rating = row.get('overall', 0.0)
            summary = str(row.get('요약', ''))[:50] + "..."

            print(f"\n[{rank}위] {c_name} ({role})")
            print(f"   • 적합도(유사도): {sim:.4f}")
            print(f"   • 실제 종합평점: {rating}점")
            print(f"   • 리뷰 요약: {summary}")
            rank += 1

    def predict_score(self, text):
        encoded = self.tokenizer.encode_plus(
            text, max_length=128, padding='max_length', truncation=True, return_tensors='pt'
        )
        input_ids = encoded['input_ids'].to(self.device)
        mask = encoded['attention_mask'].to(self.device)

        with torch.no_grad():
            scores, _ = self.model(input_ids, mask)
            scores = scores.cpu().numpy()[0]

        print(f"\n📊 '{text[:20]}...'에 대한 AI 평점 예측:")
        labels = ['복지/급여', '워라밸', '문화', '경영진', '승진', '종합']
        for label, score in zip(labels, scores):
            print(f"   - {label}: {score:.2f}점")

# ========================================================================================
# [Main Execution] 실제 사용 예시
# ========================================================================================
if __name__ == "__main__":
    # 데이터 경로: 코랩에 업로드된 엑셀 파일 우선 (/content/...)
    target_data_path = '/content/통합_리뷰_기업중복제거.xlsx'

    # 모델 경로: 코랩에 생성된 .pt 파일 경로
    target_model_path = '/content/review_rating_model.pt'

    if not os.path.exists(target_data_path) and not os.path.exists('/content/통합_리뷰_기업중복제거.xlsx - Sheet1.csv'):
         print(f"⚠️ 데이터 파일을 찾을 수 없습니다. '{target_data_path}' 경로를 확인해주세요.")

    try:
        rec_system = AIRecommender(
            model_path=target_model_path,
            data_path=target_data_path
        )

        # --- 시나리오 테스트 ---
        print("\n👇 [테스트 1] 워라밸 중시형")
        rec_system.recommend("연봉은 적어도 되니까 야근 없고 수평적인 곳", top_k=3)

        print("\n👇 [테스트 2] 성장/보상 중시형")
        rec_system.recommend("일이 힘들어도 배울게 많고 성과급 확실한 곳", top_k=3)

        # print("\n👇 [테스트 3] 점수 예측")
        # rec_system.predict_score("야근이 매일 있고 주말 출근도 강요합니다. 연봉은 최저임금 수준.")

    except Exception as e:
        print(f"\n❌ 오류 발생: {e}")
        print("1. 학습 코드를 먼저 실행하여 모델(pt 파일)을 생성했는지 확인하세요.")
        print("2. 엑셀 데이터 파일(.xlsx)이 존재하는지 확인하세요.")